# RAG Fusion Query Pipeline

This notebook shows how to implement RAG Fusion using the LlamaIndex Query Pipeline syntax.

## Setup / Load Data

We load in the pg_essay.txt data.

In [3]:
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt' -O pg_essay.txt

--2024-01-09 21:50:55--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/paul_graham/paul_graham_essay.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8000::154, 2606:50c0:8003::154, 2606:50c0:8001::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8000::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75042 (73K) [text/plain]
Saving to: ‘pg_essay.txt’

pg_essay.txt        100%[===================>]  73.28K  --.-KB/s    in 0.01s   

2024-01-09 21:50:56 (5.83 MB/s) - ‘pg_essay.txt’ saved [75042/75042]



In [7]:
from llama_index import SimpleDirectoryReader

reader = SimpleDirectoryReader(input_files=["pg_essay.txt"])
documents = reader.load_data()

### [Optional] Setup Tracing

We also setup tracing through Arize Phoenix to look at our outputs.

In [2]:
import phoenix as px

px.launch_app()
import llama_index

llama_index.set_global_handler("arize_phoenix")

/Users/jerryliu/Programming/llama-hub/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🌍 To view the Phoenix app in your browser, visit http://127.0.0.1:6006/
📺 To view the Phoenix app in a notebook, run `px.active_session().view()`
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [3]:
from llama_index.query_pipeline import (
    QueryPipeline,
    InputComponent,
    FnComponent,
    ArgPackComponent,
)
from typing import Dict, Any, List, Optional
from llama_index.llama_pack.base import BaseLlamaPack
from llama_index.llms.llm import LLM
from llama_index.llms.openai import OpenAI
from llama_index import Document, VectorStoreIndex, ServiceContext
from llama_index.response_synthesizers import TreeSummarize
from llama_index.schema import NodeWithScore
from llama_index.node_parser import SentenceSplitter


def reciprocal_rank_fusion(
    results: List[List[NodeWithScore]],
) -> List[NodeWithScore]:
    """Apply reciprocal rank fusion.

    The original paper uses k=60 for best results:
    https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf
    """
    k = 60.0  # `k` is a parameter used to control the impact of outlier rankings.
    fused_scores = {}
    text_to_node = {}

    # compute reciprocal rank scores
    for nodes_with_scores in results:
        for rank, node_with_score in enumerate(
            sorted(nodes_with_scores, key=lambda x: x.score or 0.0, reverse=True)
        ):
            text = node_with_score.node.get_content()
            text_to_node[text] = node_with_score
            if text not in fused_scores:
                fused_scores[text] = 0.0
            fused_scores[text] += 1.0 / (rank + k)

    # sort results
    reranked_results = dict(
        sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    )

    # adjust node scores
    reranked_nodes: List[NodeWithScore] = []
    for text, score in reranked_results.items():
        reranked_nodes.append(text_to_node[text])
        reranked_nodes[-1].score = score

    return reranked_nodes


llm = OpenAI(model="gpt-3.5-turbo")
chunk_sizes = [512]
query_engines = []
retrievers = {}
for chunk_size in chunk_sizes:
    splitter = SentenceSplitter(chunk_size=chunk_size, chunk_overlap=0)
    nodes = splitter.get_nodes_from_documents(documents)

    service_context = ServiceContext.from_defaults(llm=llm)
    vector_index = VectorStoreIndex(nodes, service_context=service_context)
    query_engines.append(vector_index.as_query_engine())

    retrievers[str(chunk_size)] = vector_index.as_retriever()

# define rerank component
rerank_component = FnComponent(fn=reciprocal_rank_fusion)

# construct query pipeline
p = QueryPipeline()
module_dict = {
    **retrievers,
    "input": InputComponent(),
    "summarizer": TreeSummarize(),
    # NOTE: Join args
    "join": ArgPackComponent(),
    "reranker": rerank_component,
}
p.add_modules(module_dict)
# add links from input to retriever (id'ed by chunk_size)
for chunk_size in chunk_sizes:
    p.add_link("input", str(chunk_size))
    p.add_link(str(chunk_size), "join", dest_key=str(chunk_size))
p.add_link("join", "reranker")
p.add_link("input", "summarizer", dest_key="query_str")
p.add_link("reranker", "summarizer", dest_key="nodes")

In [4]:
response = p.run(query="What did the author do during YC?")
print(str(response))

The author worked on YC, hacked, and wrote essays during their time at YC.


In [11]:
from llama_index.query_pipeline import (
    QueryPipeline,
    InputComponent,
    FnComponent,
    ArgPackComponent,
)
from typing import Dict, Any, List, Optional
from llama_index.llama_pack.base import BaseLlamaPack
from llama_index.llms.llm import LLM
from llama_index.llms.openai import OpenAI
from llama_index import Document, VectorStoreIndex, ServiceContext
from llama_index.response_synthesizers import TreeSummarize
from llama_index.schema import NodeWithScore
from llama_index.node_parser import SentenceSplitter
from llama_index.prompts import PromptTemplate


llm = OpenAI(model="gpt-3.5-turbo")
prompt_str = "Please generate a question about Paul Graham's life regarding the following topic {topic}"
prompt_tmpl = PromptTemplate(prompt_str)
splitter = SentenceSplitter(chunk_size=512, chunk_overlap=0)
nodes = splitter.get_nodes_from_documents(documents)

service_context = ServiceContext.from_defaults(llm=llm)
vector_index = VectorStoreIndex(nodes, service_context=service_context)
retriever = vector_index.as_retriever()

# construct query pipeline
p = QueryPipeline()
module_dict = {
    # "llm": llm,
    # "prompt_tmpl": prompt_tmpl,
    "input": InputComponent(),
    "summarizer": TreeSummarize(service_context=service_context),
    "retriever": retriever,
}
p.add_modules(module_dict)
# p.add_link("input", "prompt_tmpl")
# p.add_link("prompt_tmpl", "llm")
# p.add_link("llm", "retriever")
# p.add_link("llm", "summarizer", dest_key="query_str")
# p.add_link("retriever", "summarizer", dest_key="nodes")
p.add_link("input", "retriever")
p.add_link("input", "summarizer", dest_key="query_str")
p.add_link("retriever", "summarizer", dest_key="nodes")

In [12]:
# response = p.run(topic="Yc")
# print(str(response))
response = p.run(input="What did the author do during his time at YC?")
print(str(response))

The author worked on Y Combinator (YC) and had multiple responsibilities during his time there. Initially, he intended to split his time between hacking, writing essays, and working on YC. However, as YC grew and he became more excited about it, it started to take up a significant portion of his attention. He worked on selecting and helping founders, as well as writing all of YC's internal software. Additionally, he mentioned that YC's biggest source of stress for him was Hacker News (HN), which he created as a news aggregator for startup founders. Despite the stress, working on YC was engaging and allowed him to learn a lot about startups.


In [5]:
from llama_index.query_pipeline import (
    QueryPipeline,
    InputComponent,
    FnComponent,
    ArgPackComponent,
)
from typing import Dict, Any, List, Optional
from llama_index.llama_pack.base import BaseLlamaPack
from llama_index.llms.llm import LLM
from llama_index.llms.openai import OpenAI
from llama_index import Document, VectorStoreIndex, ServiceContext
from llama_index.response_synthesizers import TreeSummarize
from llama_index.schema import NodeWithScore, TextNode
from llama_index.node_parser import SentenceSplitter
from llama_index.prompts import PromptTemplate


llm = OpenAI(model="gpt-3.5-turbo")
prompt_str = "Ask if jerry likes the following {topic}"
prompt_tmpl = PromptTemplate(prompt_str)
splitter = SentenceSplitter(chunk_size=512, chunk_overlap=0)
nodes = splitter.get_nodes_from_documents(documents)

service_context = ServiceContext.from_defaults(llm=llm)
vector_index = VectorStoreIndex(nodes, service_context=service_context)
retriever = vector_index.as_retriever()

# construct query pipeline

test_node = NodeWithScore(node=TextNode(text="Jerry likes apples."))
# summarize_c = TreeSummarize(service_context=service_context).as_query_component(partial={"nodes": [test_node]})
summarize_c = TreeSummarize().as_query_component(partial={"nodes": [test_node]})
# p = QueryPipeline(chain=[summarize_c])
# response = p.run(query_str="What does jerry like")
# print(str(response))

p = QueryPipeline(chain=[prompt_tmpl, llm, summarize_c], verbose=True)
response = p.run(topic="apples")
print(str(response))

> Running module 62153dde-a31f-4566-8456-1d9d37ffda46 with input: 
topic: apples

> Running module a1ba6997-2b57-4107-b0f5-1849fb252b79 with input: 
messages: Ask if jerry likes the following apples

> Running module f1e4f64e-c253-439a-9e6d-8008bf8511be with input: 
query_str: assistant: Does Jerry like these apples?

Yes, Jerry likes these apples.


## Setup Llama Pack

Next we download the LlamaPack. All the code is in the downloaded directory - we encourage you to take a look to see the QueryPipeline syntax!

In [8]:
# Option 1: Use `download_llama_pack`
# from llama_index.llama_pack import download_llama_pack

# RAGFusionPipelinePack = download_llama_pack(
#     "RAGFusionPipelinePack",
#     "./rag_fusion_pipeline_pack",
#     skip_load=True,
#     # leave the below line commented out if using the notebook on main
#     llama_hub_url="https://raw.githubusercontent.com/run-llama/llama-hub/jerry/add_llm_compiler_pack/llama_hub"
# )

# Option 2: Import from llama_hub package
from llama_hub.llama_packs.query.rag_fusion_pipeline.base import RAGFusionPipelinePack
from llama_index.llms import OpenAI

In [9]:
pack = RAGFusionPipelinePack(documents, llm=OpenAI(model="gpt-3.5-turbo"))

## Inspecting the Code

If we take a look at how it's setup (in your downloaded directory, you'll see the following code using our QueryPipeline syntax). 

`retrievers` is a dictionary mapping a chunk size to retrievers (chunk sizes: 128, 256, 512, 1024). 

```python
# construct query pipeline
p = QueryPipeline()
module_dict = {
    **self.retrievers,
    "input": InputComponent(),
    "summarizer": TreeSummarize(),
    # NOTE: Join args
    "join": ArgPackComponent(),
    "reranker": rerank_component,
}
p.add_modules(module_dict)
# add links from input to retriever (id'ed by chunk_size)
for chunk_size in self.chunk_sizes:
    p.add_link("input", str(chunk_size))
    p.add_link(str(chunk_size), "join", dest_key=str(chunk_size))
p.add_link("join", "reranker")
p.add_link("input", "summarizer", dest_key="query_str")
p.add_link("reranker", "summarizer", dest_key="nodes")
```

We visualize the DAG below.

In [10]:
from pyvis.network import Network

net = Network(notebook=True, cdn_resources="in_line", directed=True)
net.from_nx(pack.query_pipeline.dag)
net.show("rag_dag.html")

rag_dag.html


In [11]:
response = pack.run(query="What did the author do growing up?")
print(str(response))

The author wrote short stories and tried writing programs on an IBM 1401 computer during their school days. They also mentioned painting still lives in their bedroom at night.


In [8]:
# response.source_nodes